In [1]:
cd ../dataprocessing

/Users/carverforbes/internships/google/gunshot-detection/activity-recognition/location/lbs/activity/audioset/dataprocessing


In [2]:
import audio_processing as ap

import functools
import os
import sys
from absl import logging

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.utils import np_utils
from sklearn import metrics
from sklearn.model_selection import train_test_split

/Users/carverforbes/.pyenv/versions/3.7.7/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
print(os.path.dirname(os.path.realpath('__file__')))

/Users/carverforbes/internships/google/gunshot-detection/activity-recognition/location/lbs/activity/audioset/dataprocessing


In [4]:
debug = False
logging.set_verbosity(logging.INFO)

In [5]:
src_dir = 'example_src_dir'
dest_dir = 'example_dest_dir'
# filenames should adhere to the following order
# [dataset, validation set, test set]
filenames = ['test_set2']
labels = ['Gunshot, gunfire']
features_to_extract = ['mfcc']

In [6]:
def get_dataframes():
    length = len(filenames)
    if length == 3:
        dataset_df = ap.output_df(src_dir, dest_dir, filenames[0], labels, features_to_extract)
        evaluation_df = ap.output_df(src_dir, dest_dir, filenames[1], labels, features_to_extract)
        validation_df = ap.output_df(src_dir, dest_dir, filenames[2], labels, features_to_extract)
        dfs = [dataset_df, evaluation_df, validation_df]
    elif length == 2:
        dataset_df = ap.output_df(src_dir, dest_dir, filenames[0], labels, features_to_extract)
        evaluation_df = ap.output_df(src_dir, dest_dir, filenames[1], labels, features_to_extract)
        dfs = [dataset_df, evaluation_df]
    elif length == 1:
        dataset_df = ap.output_df(src_dir, dest_dir, filenames[0], labels, features_to_extract)
        dfs = [dataset_df]
    else:
        raise ValueError('You must have at least one dataset csv and testing data csv')
    return dfs

In [ ]:
dfs = get_dataframes()

INFO:absl:The set has 13420 examples
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:a

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

...

INFO:absl:extracted features
INFO:absl:(2936, 1563.502995)
INFO:absl:extracted features
INFO:absl:(2937, 1564.057876)
INFO:absl:extracted features
INFO:absl:(2938, 1564.674666)
INFO:absl:extracted features
INFO:absl:(2939, 1565.207949)
INFO:absl:extracted features
INFO:absl:(2940, 1565.722276)
INFO:absl:extracted features
INFO:absl:(2941, 1566.325876)
INFO:absl:extracted features
INFO:absl:(2942, 1566.855793)
INFO:absl:extracted features
INFO:absl:(2943, 1567.442742)
INFO:absl:extracted features
INFO:absl:(2944, 1567.966839)
INFO:absl:extracted features
INFO:absl:(2945, 1568.554082)
INFO:absl:extracted features
INFO:absl:(2946, 1569.194009)
INFO:absl:extracted features
INFO:absl:(2947, 1569.705103)
INFO:absl:extracted features
INFO:absl:(2948, 1570.374002)
INFO:absl:extracted features
INFO:absl:(2949, 1570.923652)
INFO:absl:extracted features
INFO:absl:(2950, 1571.452885)
INFO:absl:extracted features
INFO:absl:(2951, 1572.100618)
INFO:absl:extracted features
INFO:absl:(2952, 1572.64668

INFO:absl:(3075, 1639.088739)
INFO:absl:extracted features
INFO:absl:(3076, 1639.611719)
INFO:absl:extracted features
INFO:absl:(3077, 1640.1245)
INFO:absl:extracted features
INFO:absl:(3078, 1640.614273)
INFO:absl:extracted features
INFO:absl:(3079, 1641.136944)
INFO:absl:extracted features
INFO:absl:(3080, 1641.659719)
INFO:absl:extracted features
INFO:absl:(3081, 1642.179795)
INFO:absl:extracted features
INFO:absl:(3082, 1642.715559)
INFO:absl:extracted features
INFO:absl:(3083, 1643.312871)
INFO:absl:extracted features
INFO:absl:(3084, 1643.854812)
INFO:absl:extracted features
INFO:absl:(3085, 1644.12175)
INFO:absl:extracted features
INFO:absl:(3086, 1644.627738)
INFO:absl:extracted features
INFO:absl:(3087, 1645.13223)
INFO:absl:extracted features
INFO:absl:(3088, 1645.655702)
INFO:absl:extracted features
INFO:absl:(3089, 1646.167202)
INFO:absl:extracted features
INFO:absl:(3090, 1646.687333)
INFO:absl:extracted features
INFO:absl:(3091, 1647.205341)
INFO:absl:extracted features
I

INFO:absl:extracted features
INFO:absl:(3215, 1713.810316)
INFO:absl:extracted features
INFO:absl:(3216, 1714.331412)
INFO:absl:extracted features
INFO:absl:(3217, 1714.833277)
INFO:absl:extracted features
INFO:absl:(3218, 1715.3433)
INFO:absl:extracted features
INFO:absl:(3219, 1715.860812)
INFO:absl:extracted features
INFO:absl:(3220, 1716.384103)
INFO:absl:extracted features
INFO:absl:(3221, 1716.881654)
INFO:absl:extracted features
INFO:absl:(3222, 1717.377794)
INFO:absl:extracted features
INFO:absl:(3223, 1717.873857)
INFO:absl:extracted features
INFO:absl:(3224, 1718.373602)
INFO:absl:extracted features
INFO:absl:(3225, 1719.224334)
INFO:absl:extracted features
INFO:absl:(3226, 1719.752894)
INFO:absl:extracted features
INFO:absl:(3227, 1720.258665)
INFO:absl:extracted features
INFO:absl:(3228, 1720.760291)
INFO:absl:extracted features
INFO:absl:(3229, 1721.326901)
INFO:absl:extracted features
INFO:absl:(3230, 1721.860775)
INFO:absl:extracted features
INFO:absl:(3231, 1722.388869)

INFO:absl:(3354, 1786.703632)
INFO:absl:extracted features
INFO:absl:(3355, 1787.225609)
INFO:absl:extracted features
INFO:absl:(3356, 1787.730879)
INFO:absl:extracted features
INFO:absl:(3357, 1788.255142)
INFO:absl:extracted features
INFO:absl:(3358, 1788.764721)
INFO:absl:extracted features
INFO:absl:(3359, 1789.285318)
INFO:absl:extracted features
INFO:absl:(3360, 1789.808355)
INFO:absl:extracted features
INFO:absl:(3361, 1790.342218)
INFO:absl:extracted features
INFO:absl:(3362, 1790.892632)
INFO:absl:extracted features
INFO:absl:(3363, 1791.415914)
INFO:absl:extracted features
INFO:absl:(3364, 1791.990207)
INFO:absl:extracted features
INFO:absl:(3365, 1792.526214)
INFO:absl:extracted features
INFO:absl:(3366, 1793.112797)
INFO:absl:extracted features
INFO:absl:(3367, 1793.639553)
INFO:absl:extracted features
INFO:absl:(3368, 1794.166953)
INFO:absl:extracted features
INFO:absl:(3369, 1794.690069)
INFO:absl:extracted features
INFO:absl:(3370, 1795.2064)
INFO:absl:extracted features

INFO:absl:extracted features
INFO:absl:(3494, 1861.359235)
INFO:absl:extracted features
INFO:absl:(3495, 1861.943251)
INFO:absl:extracted features
INFO:absl:(3496, 1862.476725)
INFO:absl:extracted features
INFO:absl:(3497, 1863.014415)
INFO:absl:extracted features
INFO:absl:(3498, 1863.54696)
INFO:absl:extracted features
INFO:absl:(3499, 1864.080388)
INFO:absl:extracted features
INFO:absl:(3500, 1864.652625)
INFO:absl:extracted features
INFO:absl:(3501, 1865.17674)
INFO:absl:extracted features
INFO:absl:(3502, 1865.712471)
INFO:absl:extracted features
INFO:absl:(3503, 1866.266908)
INFO:absl:extracted features
INFO:absl:(3504, 1866.79257)
INFO:absl:extracted features
INFO:absl:(3505, 1867.335266)
INFO:absl:extracted features
INFO:absl:(3506, 1867.8907)
INFO:absl:extracted features
INFO:absl:(3507, 1868.402999)
INFO:absl:extracted features
INFO:absl:(3508, 1868.982685)
INFO:absl:extracted features
INFO:absl:(3509, 1869.510204)
INFO:absl:extracted features
INFO:absl:(3510, 1870.021097)
IN

In [ ]:
dataset_df = dfs[0]
dataset_df.head()

Convert features and classification labels into numpy arrays

In [ ]:
X = np.array(dataset_df.mfcc.tolist(), dtype=object)
y = np.array(dataset_df.label.tolist())

In [ ]:
def get_data_for_model(dfs, ratio):
    length = len(dfs)
    if length == 3:
        train_x = np.array(dfs[0].mfcc.tolist(), dtype=object)
        train_y = np.array(dfs[0].label.tolist())
        test_x = np.array(dfs[1].mfcc.tolist(), dtype=object)
        test_y = np.array(dfs[1].label.tolist())
        val_x = np.array(dfs[2].mfcc.tolist(), dtype=object)
        val_y = np.array(dfs[2].label.tolist())
        return train_x, train_y, val_x, val_y, test_x, test_y
    elif length == 2:
        train_x = np.array(dfs[0].mfcc.tolist(), dtype=object)
        train_y = np.array(dfs[0].label.tolist())
        test_x = np.array(dfs[1].mfcc.tolist(), dtype=object)
        test_y = np.array(dfs[1].label.tolist())
        return train_x, train_y, test_x, test_y
    elif length == 1:
        X = np.array(dataset_df.mfcc.tolist(), dtype=object)
        y = np.array(dataset_df.label.tolist())
        train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.20, random_state = 42)
        return train_x, train_y, test_x, test_y

# Split the dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 42)

In [ ]:
# trying to fix bug:
# ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).
# It worked!!!
from keras import backend as K
x_train = K.cast_to_floatx(x_train)
y_train = K.cast_to_floatx(y_train)
x_test = K.cast_to_floatx(x_test)
y_test = K.cast_to_floatx(y_test)

In [ ]:
data = (x_train, x_test, y_train, y_test)

In [ ]:
def visualize_training(history, filename):
    history_dict = history.history
    history_dict.keys()
    acc = history_dict['accuracy']
#     val_acc = history_dict['val_accuracy']
    loss = history_dict['loss']
#     val_loss = history_dict['val_loss']

    epochs = range(1, len(acc) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,7))
    
    # "bo" is for "blue dot"
    ax1.plot(epochs, loss, 'bo', label='Training loss')
    # b is for "solid blue line"
#     ax1.plot(epochs, val_loss, 'b', label='Validation loss')
    ax1.set_title('Training and validation loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    
    ax2.plot(epochs, acc, 'bo', label='Training acc')
#     ax2.plot(epochs, val_acc, 'b', label='Validation acc')
    ax2.set_title('Training and validation accuracy')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend(loc='lower right')
    
    plt.savefig(filename, bbox_inches='tight')
    plt.show()


In [ ]:
def model_config1(activation, optimizer, metrics):
    model = keras.Sequential([
        keras.Input(shape=(20,)),
        keras.layers.Dense(20, activation=activation),
        keras.layers.Dense(1, activation=activation)
    ])
    model.compile(optimizer=optimizer,
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=metrics)
    return model

In [ ]:
def model_train1(model, data, epochs):
    x_train, x_test, y_train, y_test = data
    history = model.fit(x_train,
                        y_train,
                        epochs=epochs,
                        verbose=1,
                        use_multiprocessing=False
    )
    return history

In [ ]:
def print_performance(history):
    loss = history.history.get('loss')
    accuracy = history.history.get('accuracy')
    tp = history.history.get('tp')
    fp = history.history.get('fp')
    tn = history.history.get('tn')
    fn = history.history.get('fn')
    print('Loss: {}'.format(loss))
    print('accuracy: {}'.format(accuracy))
    print('True Positives: {}'.format(tp))
    print('False Positives: {}'.format(fp))
    print('True Negatives: {}'.format(tn))
    print('False Negatives: {}'.format(fn))
    prec = []
    rec = []
    f1 = []
    for tp1, fp1 in list(zip(tp, fp)):
        prec.append(precision(tp1, fp1))
    for tp1, fn1 in list(zip(tp, fn)):
        rec.append(recall(tp1, fn1))
    for rec1, prec1 in list(zip(rec, prec)):
        f1.append(f1score(rec1, prec1))
        print('Recall: {}'.format(rec))
    print('Precision: {}'.format(prec))
    print('F1-Score: {}'.format(f1))

In [ ]:
def precision(tp, fp):
    sum = tp + fp
    if sum == 0:
        return 0
    return tp / sum

In [ ]:
def recall(tp, fn):
    sum = tp + fn
    if sum == 0:
        return 0
    return tp / sum

In [ ]:
def f1score(recall, precision):
    sum = recall + precision
    if sum == 0:
        return 0
    return 2 * recall * precision / sum

In [ ]:
path = os.path.join(dest_dir, 'results1')
metrics_list = [['accuracy'], 
                [tf.keras.metrics.TruePositives(name='tp')], 
                [tf.keras.metrics.TrueNegatives()], 
                [tf.keras.metrics.FalseNegatives()], 
                [tf.keras.metrics.FalsePositives()]
               ]
metrics = [
    'accuracy',
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
]
model = model_config1('softmax', 'adam', metrics)
history = model_train1(model, data, 20)
visualize_training(history, path)
print_performance(history)